In [ ]:
!nvidia-smi

Fri Feb 19 07:47:02 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.39       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   57C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -qU wandb
!wandb login 984196dfc7bc6ae6093fed3667fd5da413300d29

     |████████████████████████████████| 2.0MB 17.6MB/s 
     |████████████████████████████████| 102kB 14.5MB/s 
     |████████████████████████████████| 133kB 59.3MB/s 
     |████████████████████████████████| 163kB 64.8MB/s 
     |████████████████████████████████| 71kB 11.4MB/s 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
# @title Download dataset
import os
import shutil
import time
from google_drive_downloader import GoogleDriveDownloader as gdd

if not os.path.exists("/content/valid.zip"):
    # https://drive.google.com/file/d/1ux-mv7250XsgZo-vsk4QIedfGbELl9oZ/view?usp=sharing
    # LR - 64x64 images
    ! gdown --id 1ux-mv7250XsgZo-vsk4QIedfGbELl9oZ
    !unzip -q valid.zip -d /content/images
    # gdd.download_file_from_google_drive(file_id='1ux-mv7250XsgZo-vsk4QIedfGbELl9oZ',
    #                                     dest_path='./images/valid.zip',
    #                                     unzip=True,
    #                                     showsize=True,
    #                                     )
    # !rm -rf /content/images/valid.zip

# download scripts
!rm -rf *.py
time.sleep(2)
!wget -q https://raw.githubusercontent.com/veb-101/Esrgan-pytorch/master/trainer.py -O trainer.py
!wget -q https://raw.githubusercontent.com/veb-101/Esrgan-pytorch/master/utils.py -O utils.py
!wget -q https://raw.githubusercontent.com/veb-101/Esrgan-pytorch/master/models.py -O models.py

# # https://drive.google.com/file/d/1ak35jClzM1kAf7p4325b5P9CIudfuwWq/view?usp=sharing

# # if not os.path.exists("/content/pretrained.zip"):
# #     # https://drive.google.com/file/d/1ak35jClzM1kAf7p4325b5P9CIudfuwWq/view?usp=sharing
# #     # HR - 256x256 images
# #     gdd.download_file_from_google_drive(file_id='1ak35jClzM1kAf7p4325b5P9CIudfuwWq',
# #                                         dest_path='./pretrained.zip',
# #                                         unzip=True,
# #                                         showsize=True,
# #                                         )


if not os.path.exists("/content/train.zip"):
    # https://drive.google.com/file/d/1RGvBO7wVCI4aPNkjy8w-Sl3Kzp2UCqXm/view?usp=sharing
    # HR - 256x256 images
    ! gdown --id 1RGvBO7wVCI4aPNkjy8w-Sl3Kzp2UCqXm
    !unzip -q train.zip -d /content/images
    # gdd.download_file_from_google_drive(file_id='1RGvBO7wVCI4aPNkjy8w-Sl3Kzp2UCqXm',
    #                                     dest_path='./images/train.zip',
    #                                     unzip=True,
    #                                     showsize=True,
    #                                     )
#     # !rm -rf /content/images/train.zip

Downloading...
From: https://drive.google.com/uc?id=1ux-mv7250XsgZo-vsk4QIedfGbELl9oZ
To: /content/valid.zip
8.33MB [00:00, 22.9MB/s]
Downloading...
From: https://drive.google.com/uc?id=1RGvBO7wVCI4aPNkjy8w-Sl3Kzp2UCqXm
To: /content/train.zip
1.47GB [00:19, 77.3MB/s]


In [ ]:
#@title Setup

import trainer
import os
import random
from PIL import Image
import numpy as np

import torch
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms.functional as TF
import importlib

import warnings
warnings.simplefilter("ignore", UserWarning)
warnings.simplefilter("ignore", RuntimeWarning)

seed = 41
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)


def get_default_device():
    """Pick GPU if available, else CPU"""
    if torch.cuda.is_available():
        return torch.device("cuda")
    else:
        return torch.device("cpu")


device = get_default_device()


class ESR_Dataset(Dataset):
    def __init__(self, num_images=9000, path=r"images", train=True):
        self.path = path
        self.is_train = train

        if not os.path.exists(self.path):
            raise Exception(f"[!] dataset is not exited")

        self.image_paths = os.listdir(os.path.join(self.path, "hr"))

        if num_images == -1:
            num_images = len(self.image_paths)

        
        self.image_file_name = np.random.choice(self.image_paths, size=num_images, replace=False)

        if not train:
            self.image_file_name = sorted(self.image_file_name)
        

        self.mean = np.array([0.485, 0.456, 0.406])
        self.std = np.array([0.229, 0.224, 0.225])
        
    def __getitem__(self, item):
        file_name = self.image_file_name[item]
        high_resolution = Image.open(os.path.join(self.path, "hr", file_name)).convert(
            "RGB"
        )
        low_resolution = Image.open(os.path.join(self.path, "lr", file_name)).convert(
            "RGB"
        )

        if self.is_train:
            if random.random() > 0.5:
                high_resolution = TF.vflip(high_resolution)
                low_resolution = TF.vflip(low_resolution)

            if random.random() > 0.5:
                high_resolution = TF.hflip(high_resolution)
                low_resolution = TF.hflip(low_resolution)

            if random.random() > 0.5:
                high_resolution = TF.rotate(high_resolution, 90)
                low_resolution = TF.rotate(low_resolution, 90)

        high_resolution = TF.to_tensor(high_resolution)
        low_resolution = TF.to_tensor(low_resolution)

        high_resolution = TF.normalize(high_resolution, self.mean, self.std)
        low_resolution = TF.normalize(low_resolution, self.mean, self.std)

        images = {"lr": low_resolution, "hr": high_resolution}

        return images

    def __len__(self):
        return len(self.image_file_name)


def make_dataloaders(
    batch_size=32, n_workers=4, shuffle=True, **kwargs):  # A handy function to make our dataloaders
    dataset = ESR_Dataset(**kwargs)

    pin = torch.cuda.is_available()

    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        num_workers=n_workers,
        pin_memory=pin,
        shuffle=shuffle,
    )
    return dataloader

In [ ]:
config = {
    "image_size": 256,
    "batch_size": 16,
    "start_epoch": 1,
    "num_epoch": 100,
    "sample_batch_size": 1,
    "checkpoint_dir": "./checkpoints",
    "sample_dir": "./samples",
    "workers": 6,
    "scale_factor": 4,
    "num_rrdn_blocks": 23,
    "nf": 64,
    "gc": 32,
    "b1": 0.9,
    "b2": 0.999,
    "weight_decay": 1e-2,
    # ------ PSNR ------
    "p_lr": 2e-4,
    "p_decay_iter": [25, 50, 75],
    "p_perceptual_loss_factor": 0,
    "p_adversarial_loss_factor": 0,
    "p_content_loss_factor": 1,
    # ------------------
    # ------ ADVR ------
    "g_lr": 1e-4,
    "g_decay_iter": [40, 80, 120],
    "g_perceptual_loss_factor": 1,
    "g_adversarial_loss_factor": 5e-2,
    "g_content_loss_factor": 0.5,
    # ------------------
    "is_psnr_oriented": True,
    "load_previous_opt": True,
}

In [ ]:
#@title Create Dataloader
training_images =  9000#@param {type:"integer"}
validation_images =  -1#@param {type:"integer"}


esr_dataloader_train = make_dataloaders(batch_size=config["batch_size"] + 2, 
                                        n_workers=config["workers"], 
                                        shuffle=True, 
                                        num_images=training_images, 
                                        path=r"./images/train", 
                                        train=True)

esr_dataloader_val = make_dataloaders(batch_size=config["batch_size"],
                                      n_workers=config["workers"], 
                                      shuffle=False, 
                                      num_images=validation_images, 
                                      path=r"./images/valid", 
                                      train=False)

In [ ]:
#@title reload
import trainer
import models
import utils

importlib.reload(utils)
importlib.reload(models)
importlib.reload(trainer)

if not os.path.exists(config["sample_dir"]):
    os.makedirs(config["sample_dir"])

In [ ]:
import wandb
run = wandb.init(project="esrgan-pytorch")

wandb: W&B API key is configured (use `wandb login --relogin` to force relogin)


In [ ]:
config["start_epoch"] = 116
config["num_epoch"] = 200 - config["start_epoch"] + 1

config["num_rrdn_blocks"] = 23
config["nf"] = 64

config['p_decay_iter'] = [25, 55, 80]
config['g_decay_iter'] = [50, 100, 150]

config["is_psnr_oriented"] = False
config["load_previous_opt"] = True

hyperparams = wandb.config
hyperparams.adv_factor = config["g_adversarial_loss_factor"]
hyperparams.per_loss = config["g_content_loss_factor"]
hyperparams.epochs = config["num_epoch"]
hyperparams.RRDN = config["num_rrdn_blocks"]
hyperparams.nf = config["nf"]
hyperparams.psnr_oriented = config["is_psnr_oriented"]
hyperparams.p_decay_iter = config["p_decay_iter"]
hyperparams.g_decay_iter = config["g_decay_iter"]

    Bicubic Ups: PSNR: 25.51, SSIM: 0.7185, FID: 0.9238


* **Warmup Training EPOCH 100 - PSNR: 26.463, SSIM: 0.7445, , FID: 0.6501**
    
----------------------------------------

* Epoch: 19 -> 

Dis loss: 0.4575 Gen loss: 1.3375 Per loss:: 1.2368 Adv loss:: 0.0824 Con loss:: 0.0183,
Validation Set: PSNR: 24.914, SSIM: 0.7012, FID: 0.6342

Bicubic Ups: PSNR: 25.51, SSIM: 0.7185, FID: 0.9238

Best val scores  till epoch 19 -> PSNR: 24.914, SSIM: 0.7012, , FID: 0.6342


* Epoch: 25 -> 
Dis loss: 0.1962 Gen loss: 1.4147 Per loss:: 1.2266 Adv loss:: 0.1701 Con loss:: 0.018

Validation Set: PSNR: 24.5728, SSIM: 0.6982, FID: 0.6209

Bicubic Ups: PSNR: 25.51, SSIM: 0.7185, FID: 0.9238

Best val scores  till epoch 25 -> PSNR: 24.914, SSIM: 0.7012, , FID: 0.6209

In [ ]:
#@title training

import gc

torch.cuda.empty_cache()
gc.collect()

for key, value in config.items():
    print(f"{key:30}: {value}")

print("\n\n")
print(f"ESRGAN start")

model = trainer.Trainer(
    config, esr_dataloader_train, esr_dataloader_val, device)
model_metrics = model.train()

image_size                    : 256
batch_size                    : 16
start_epoch                   : 116
num_epoch                     : 85
sample_batch_size             : 1
checkpoint_dir                : ./checkpoints
sample_dir                    : ./samples
workers                       : 6
scale_factor                  : 4
num_rrdn_blocks               : 23
nf                            : 64
gc                            : 32
b1                            : 0.9
b2                            : 0.999
weight_decay                  : 0.01
p_lr                          : 0.0002
p_decay_iter                  : [25, 55, 80]
p_perceptual_loss_factor      : 0
p_adversarial_loss_factor     : 0
p_content_loss_factor         : 1
g_lr                          : 0.0001
g_decay_iter                  : [50, 100, 150]
g_perceptual_loss_factor      : 1
g_adversarial_loss_factor     : 0.05
g_content_loss_factor         : 0.5
is_psnr_oriented              : False
load_previous_opt             : Tru

Downloading: "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to /root/.cache/torch/hub/checkpoints/vgg19-dcbb9e9d.pth


Downloading: "https://download.pytorch.org/models/inception_v3_google-1a9a5a14.pth" to /root/.cache/torch/hub/checkpoints/inception_v3_google-1a9a5a14.pth


[Epoch 116/200] [Batch 1/500][D loss 0.0498] [G loss 1.39582][perceptual loss 1.111][adversarial loss 0.1897][content loss 0.0952]
[Epoch 116/200] [Batch 251/500][D loss 0.04868] [G loss 1.55479][perceptual loss 1.2862][adversarial loss 0.1911][content loss 0.0775]
[Epoch 116/200] [Batch 500/500][D loss 0.04911] [G loss 1.4225][perceptual loss 1.1424][adversarial loss 0.1906][content loss 0.0895]

Epoch: 116 -> Dis loss: 0.0489 Gen loss: 1.402 Per loss:: 1.1325 Adv loss:: 0.1909 Con loss:: 0.0786
Validation Set: PSNR: 25.4675, SSIM: 0.7125, FID: 0.5458
Bicubic Ups: PSNR: 25.51, SSIM: 0.7185, FID: 0.9238
Best val scores  till epoch 116 -> PSNR: 25.4675, SSIM: 0.7125, , FID: 0.5458


[Epoch 117/200] [Batch 1/500][D loss 0.0489] [G loss 1.14065][perceptual loss 0.8746][adversarial loss 0.191][content loss 0.075]
[Epoch 117/200] [Batch 251/500][D loss 0.04867] [G loss 1.18051][perceptual loss 0.9286][adversarial loss 0.1912][content loss 0.0607]
[Epoch 117/200] [Batch 500/500][D loss 0.04881] [G loss 1.25421][perceptual loss 0.9948][adversarial loss 0.1913][content loss 0.068]

Epoch: 117 -> Dis loss: 0.0489 Gen loss: 1.402 Per loss:: 1.1325 Adv loss:: 0.1909 Con loss:: 0.0786
Validation Set: PSNR: 25.443, SSIM: 0.7115, FID: 0.5433
Bicubic Ups: PSNR: 25.51, SSIM: 0.7185, FID: 0.9238
Best val scores  till epoch 117 -> PSNR: 25.4675, SSIM: 0.7125, , FID: 0.5433


...

[Epoch 141/200] [Batch 1/500][D loss 0.04916] [G loss 1.45079][perceptual loss 1.1777][adversarial loss 0.1908][content loss 0.0823]
[Epoch 141/200] [Batch 251/500][D loss 0.04869] [G loss 1.3761][perceptual loss 1.0932][adversarial loss 0.1914][content loss 0.0914]
[Epoch 141/200] [Batch 500/500][D loss 0.04914] [G loss 1.18284][perceptual loss 0.9353][adversarial loss 0.1906][content loss 0.057]

Epoch: 141 -> Dis loss: 0.0489 Gen loss: 1.3948 Per loss:: 1.1251 Adv loss:: 0.191 Con loss:: 0.0787
Validation Set: PSNR: 25.4847, SSIM: 0.713, FID: 0.5386
Bicubic Ups: PSNR: 25.51, SSIM: 0.7185, FID: 0.9238
Best val scores  till epoch 141 -> PSNR: 25.4945, SSIM: 0.713, , FID: 0.5357


[Epoch 142/200] [Batch 1/500][D loss 0.04859] [G loss 1.32281][perceptual loss 1.07][adversarial loss 0.1913][content loss 0.0616]
[Epoch 142/200] [Batch 251/500][D loss 0.04857] [G loss 1.52425][perceptual loss 1.2578][adversarial loss 0.1912][content loss 0.0752]
[Epoch 142/200] [Batch 500/500][D loss 0.04896] [G loss 1.12094][perceptual loss 0.8839][adversarial loss 0.1911][content loss 0.0459]

Epoch: 142 -> Dis loss: 0.0489 Gen loss: 1.3926 Per loss:: 1.123 Adv loss:: 0.191 Con loss:: 0.0786
Validation Set: PSNR: 25.48, SSIM: 0.7128, FID: 0.5413
Bicubic Ups: PSNR: 25.51, SSIM: 0.7185, FID: 0.9238


[Epoch 143/200] [Batch 1/500][D loss 0.04874] [G loss 1.47848][perceptual loss 1.2165][adversarial loss 0.1915][content loss 0.0704]
[Epoch 143/200] [Batch 251/500][D loss 0.04858] [G loss 1.60265][perceptual loss 1.3162][adversarial loss 0.1912][content loss 0.0952]
[Epoch 143/200] [Batch 500/500][D loss 0.04967] [G loss 1.49594][perceptual loss 1.2127][adversarial loss 0.1899][content loss 0.0933]

Epoch: 143 -> Dis loss: 0.0489 Gen loss: 1.3934 Per loss:: 1.1238 Adv loss:: 0.191 Con loss:: 0.0786
Validation Set: PSNR: 25.4845, SSIM: 0.7125, FID: 0.5391
Bicubic Ups: PSNR: 25.51, SSIM: 0.7185, FID: 0.9238


[Epoch 144/200] [Batch 1/500][D loss 0.04954] [G loss 1.24779][perceptual loss 0.9819][adversarial loss 0.1904][content loss 0.0754]


KeyboardInterrupt: ignored

In [ ]:
# 79 new adv loss, con loss best

In [ ]:
# !zip -rq validation_images_till_76.zip /content/samples
# !rsync /content/validation_images_till_76.zip /content/drive/MyDrive/Project-ESRGAN/validation_images_till_76.zip 

In [ ]:
run.finish()

In [ ]:
#@title Testing Images

import cv2
from google.colab.patches import cv2_imshow
import numpy as np
import models
import importlib
importlib.reload(models)

checkpoint_number = 9

checkpoint = torch.load(rf"/content/checkpoint_{checkpoint_number}.tar")
Model = models.Generator(channels=3, nf=32, gc=32, num_res_blocks=11, scale=4)
Model.load_state_dict(checkpoint[rf"generator_dict_{checkpoint_number}"])
Model.to(device)
# print("Generator weights loaded.")


image_path = r"/content/images/lr/00000.png"


def test_image(image_path, model):

    image = cv2.imread(image_path, cv2.IMREAD_COLOR)
    cv2_imshow(image)
    image = image // 255.0
    image= image.astype(np.float32)

    image = np.moveaxis(image, (0, 1, 2), (1, 2, 0))
    image = torch.from_numpy(np.expand_dims(image, 0))
    with torch.no_grad():
        # with torch.cuda.amp.autocast():
        high_res = model(image.to(device))
    
    high_res = high_res.cpu().detach().permute(0, 2, 3, 1).numpy()
    cv2_imshow(high_res[0] * 255.0)
    print(high_res.shape)

test_image(image_path, Model)


# Model.eval()
# with torch.no_grad():
# with torch.cuda.amp.autocast(enabled=False):
    # out = Model(image)


Run js function in console
```
function KeepClicking(){
console.log("Clicking");
document.querySelector("colab-connect-button").click()
}
setInterval(KeepClicking, 480000)
```

In [ ]:
# !rm -rf /content/samples
# !rm -rf /content/drive/MyDrive/Project-ESRGAN
# !rm -rf checkpoint*